In [ ]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import fiona
from shapely.ops import cascaded_union
import tqdm

In [ ]:

input_raster = r"E:\rastermunicpial\atlacomulco\atlacomulco_elevacion.tif"
input_shapefile =r"E:\rastermunicpial\atlacomulco\Atlacomulco_casas_limpio_id.shp"


In [ ]:
shp_deteccion=gpd.read_file(input_shapefile)
Zs=[]
with fiona.open(input_shapefile, 'r') as shapefile:

    for i,feature in tqdm.tqdm(enumerate(shapefile),total=len(shapefile)):
        shapes=[feature["geometry"]]
#         if i==0:
        with rasterio.open(input_raster, 'r') as src:
            out_image, out_transform = mask(src, shapes, crop=True) # setting all pixels outside of the feature zone to zero
            out_meta = src.meta
            img=out_image[0,:,:]
            Z=img[img!=0].mean()
            Zs.append(Z)

shp_deteccion["Z"]=Zs
shp_deteccion.fillna(3).to_file(r"E:\rastermunicpial\iztapan_sal/test.shp")


In [ ]:

def cortar_deteccion_municipio(shp_municipio,shp_deteccion,output_path,buffer=20):
    """
    Funcion para cortar las detecciones unicamente dentro de municipo 
    """
    municipio=gpd.read_file(shp_municipio)
    municipio=municipio.to_crs(3857)
    municipio_r=cascaded_union(municipio["geometry"].buffer(buffer))
    deteccion=gpd.read_file(shp_deteccion)
    deteccion=deteccion.to_crs(3857)
    deteccionsobremun=deteccion[[municipio_r.contains(x) for x in deteccion["geometry"].centroid]]
    deteccionsobremun.to_file(output_path)


In [ ]:


shp_municipio=r"C:/Users/ASUS/Documents/GobiernoEdomex/Agua/geoshapes/Full_manzanas/Manzana_Ixtapan_Sal.shp"
shp_deteccion=r"E:\rastermunicpial\iztapan_sal/test.shp"
output_path=r"E:\rastermunicpial\iztapan_sal\Ixtapan_casas_mun_Z.shp"
cortar_deteccion_municipio(shp_municipio,shp_deteccion,output_path,buffer=20)

In [ ]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import fiona
from shapely.ops import cascaded_union
import tqdm

def cortar_deteccion_municipio(shp_municipio,shp_deteccion,output_path,buffer=20):
    """
    Funcion para cortar las detecciones unicamente dentro de municipo 
    """
    municipio=gpd.read_file(shp_municipio)
    municipio=municipio.to_crs(3857)
    municipio_r=cascaded_union(municipio["geometry"].buffer(buffer))
    deteccion=gpd.read_file(shp_deteccion)
    deteccion=deteccion.to_crs(3857)
    deteccionsobremun=deteccion[[municipio_r.contains(x) for x in deteccion["geometry"].centroid]]
    deteccionsobremun.to_file(output_path)


path=r"E:\rastermunicpial\atlacomulco"
input_raster = path+"/atlacomulco_elevacion.tif"
input_shapefile =path+"/Atlacomulco_casas_limpio_id.shp"

shp_deteccion=gpd.read_file(input_shapefile)
Zs=[]
with fiona.open(input_shapefile, 'r') as shapefile:
    for i,feature in tqdm.tqdm(enumerate(shapefile),total=len(shapefile)):
        shapes=[feature["geometry"]]
        with rasterio.open(input_raster, 'r') as src:
            out_image, out_transform = mask(src, shapes, crop=True) # setting all pixels outside of the feature zone to zero
            out_meta = src.meta
            img=out_image[0,:,:]
            Z=img[img!=0].mean()
            Zs.append(Z)


shp_deteccion["Z"]=Zs
shp_deteccion.fillna(3).to_file(path+"/test.shp")

shp_municipio=r"C:/Users/ASUS/Documents/GobiernoEdomex/Agua/geoshapes/Full_manzanas/Manzana_Atlacomulco.shp"
shp_deteccion=path+"/test.shp"
output_path=path+"/Atlacomulco_casasZ_mun_.shp"
cortar_deteccion_municipio(shp_municipio,shp_deteccion,output_path,buffer=20)